In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

In [ ]:
train=pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')

In [ ]:
train.head()

In [ ]:
unique_stock_id=train.stock_id.unique()

In [ ]:
id_book_count=pd.DataFrame(train[['stock_id']].value_counts().sort_index(),columns=['count'])
id_book_count.reset_index().style.background_gradient(subset=['count'])

In [ ]:
# when does trade happen- when the buyer and seller at the same price point
# o
# average increase per second
# average drop in value per second
# spread of ask vs call
# number of trades that took place

In [ ]:
def wap(df):
    a1=(df['bid_price1']*df['ask_size1']+df['ask_price1']*df['bid_size1'])/(df['bid_size1']+df['ask_size1'])
    a2=(df['bid_price2']*df['ask_size2']+df['ask_price2']*df['bid_size2'])/(df['bid_size2']+df['ask_size2'])
    return a1,a2
def log_returns(series):
    return np.log(series).diff()
def realized_volatility(series):
    return np.sqrt(np.sum(np.power(series,2)))

In [ ]:

def book_order_preprocessing():
    book_stacked=pd.DataFrame([])
    for i in unique_stock_id:
        stock_folder='../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id='+str(i)+'/*'
        filepath=glob.glob(stock_folder)[0]
        book=pd.read_parquet(filepath)
        wap_req=['bid_price1', 'ask_price1',
           'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2',
           'ask_size2']
        book['wap1']=(book['bid_price1']*book['ask_size1']+book['ask_price1']*book['bid_size1'])/(book['bid_size1']+book['ask_size1'])
        book['wap2']=(book['bid_price2']*book['ask_size2']+book['ask_price2']*book['bid_size2'])/(book['bid_size2']+book['ask_size2'])
        book['log_return1']=book.groupby(['time_id'])['wap1'].apply(log_returns)
        book['log_return2']=book.groupby(['time_id'])['wap2'].apply(log_returns)
        book['bid_ask_spread1']=book['ask_price1']/book['bid_price1']-1
        book['bid_ask_spread2']=book['ask_price2']/book['bid_price2']-1
        book['ask_per_bid_size1']=book['ask_size1']/(book['bid_size1']+1)
        book['ask_per_bid_size2']=book['ask_size2']/(book['bid_size2']+1)
        book['rise_ask1']=book.groupby(['time_id'])['ask_price1'].diff().map(lambda x: 1 if x>0 else 0)
        book['rise_ask2']=book.groupby(['time_id'])['ask_price2'].diff().map(lambda x: 1 if x>0 else 0)
        data_stock=book.groupby('time_id').mean()
        book['historic_volatility']=np.power(id_0['log_returns'],2)
        data_stock=pd.merge(data_stock,pd.DataFrame(np.sqrt(book.groupby('time_id',)['historic_volatility'].sum())).reset_index(),on='time_id').copy()
        data_stock['stock_id']=i
        book_stacked=pd.concat([book_stacked,data_stock])
    book_stacked.to_parquet('data.parquet',index=False)

In [ ]:
trade=pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0/ef805fd82ff54fadb363094e3b122ab9.parquet')

In [ ]:
trade.head()

In [ ]:
def trade_order_preprocessing():
    book_stacked=pd.DataFrame([])
    for i in unique_stock_id:
        print(f'sock_id{i}')
        stock_folder='../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id='+str(i)+'/*'
        filepath=glob.glob(stock_folder)[0]
        trade=pd.read_parquet(filepath)
        trade.drop(['seconds_in_bucket'],inplace=True,axis=1)
        d=trade.groupby(['time_id'])[['size','order_count']].sum().reset_index()
        d2=trade.groupby(['time_id'])[['price']].min().reset_index()
        d=pd.merge(d,d2,on='time_id')
        d2=trade.groupby(['time_id'])[['price']].max().reset_index()
        d=pd.merge(d,d2,on='time_id')
        d2=trade.groupby(['time_id']).std().reset_index()
        d=pd.merge(d,d2,on='time_id')
        d['stock_id']=i
        book_stacked=pd.concat([book_stacked,d])
    book_stacked.to_parquet('data1.parquet',index=False)

In [ ]:
trade_order_preprocessing()